# Numerai Modeling: Feature Engineering, Ensembling, and Advanced Training

This notebook demonstrates several advanced modeling techniques for the Numerai tournament:
1. **Feature Engineering**: Creating new features using UMAP, Denoising Autoencoders, Contrastive Learning, and CTGAN.
2. **Target Exploration**: Analyzing auxiliary targets.
3. **Base Model Training**: Training LightGBM models on different targets and engineered features.
4. **Stacked Ensembling**: Combining base model predictions using a meta-model.
5. **Era-Invariant Training**: Using a PyTorch MLP with custom loss functions (correlation, era variance penalty, feature exposure penalty).
6. **Model Selection & Upload**: Choosing the final model and preparing for submission.

In [1]:
# Install dependencies
!pip3 install -q numerapi pandas pyarrow matplotlib lightgbm scikit-learn cloudpickle==2.2.1 seaborn scipy==1.10.1 umap-learn tensorflow torch ctgan tqdm

# Inline plots
%matplotlib inline

## Configuration

In [2]:
import pandas as pd
import numpy as np
import json
import gc
from numerapi import NumerAPI
import lightgbm as lgb
from sklearn.model_selection import GroupKFold
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler
import cloudpickle
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import pearsonr
from tqdm.notebook import tqdm

# --- Configuration ---
DATA_VERSION = "v5.0"
MAIN_TARGET = "target_cyrusd_20"
AUX_TARGETS = [
  "target_victor_20",
  "target_xerxes_20",
  "target_teager2b_20"
]
TARGET_CANDIDATES = [MAIN_TARGET] + AUX_TARGETS
ERA_COL = "era"
DATA_TYPE_COL = "data_type"
TARGET_COL = "target" # Alias for MAIN_TARGET in original notebook
PREDICTION_COL = "prediction"

# Feature Engineering Hyperparameters
UMAP_N_COMPONENTS = 50
AE_ENCODING_DIM = 64
CONTRASTIVE_EMB_DIM = 64
CTGAN_EPOCHS = 100 # Reduced for speed in example

# Stacking Ensemble Config
N_FOLDS = 5 # Number of folds for OOF predictions
STACKING_MODEL_TYPE = 'LGBM' # 'LGBM' or 'Linear'

# PyTorch MLP Config
MLP_EPOCHS = 10 # Reduced for speed in example
MLP_BATCH_SIZE = 512
MLP_LR = 0.001
VARIANCE_PENALTY_WEIGHT = 0.01 # lambda1
FEATURE_EXPOSURE_WEIGHT = 0.01 # lambda2
TOP_N_FEATURES_FOR_EXPOSURE = 50 # Use top N features for exposure penalty

# Model Selection Flags
USE_STACKING = True # Set to True to use Stacking, False for MLP
USE_MLP = False

# Downsampling for faster execution (optional)
DOWNSAMPLE_TRAIN_ERAS = 4 # Use every Nth era for training
DOWNSAMPLE_VALID_ERAS = 4 # Use every Nth era for validation

pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', lambda x: f'{x:.6f}')

OSError: dlopen(/Users/I570611/Documents/GitHub/BarrierOptions/.conda/lib/python3.11/site-packages/lightgbm/lib/lib_lightgbm.dylib, 0x0006): Library not loaded: @rpath/libomp.dylib
  Referenced from: <8FC36893-94B8-343C-9D9F-4CCBFE81B89B> /Users/I570611/Documents/GitHub/BarrierOptions/.conda/lib/python3.11/site-packages/lightgbm/lib/lib_lightgbm.dylib
  Reason: tried: '/usr/local/opt/libomp/lib/libomp.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/usr/local/opt/libomp/lib/libomp.dylib' (no such file), '/opt/homebrew/opt/libomp/lib/libomp.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/opt/homebrew/opt/libomp/lib/libomp.dylib' (no such file), '/opt/local/lib/libomp/libomp.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/opt/local/lib/libomp/libomp.dylib' (no such file), '/usr/local/opt/libomp/lib/libomp.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/usr/local/opt/libomp/lib/libomp.dylib' (no such file), '/opt/homebrew/opt/libomp/lib/l

## 1. Feature Engineering Functions

Define functions to generate new features.

In [ ]:
import umap
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from ctgan import CTGAN
from sklearn.preprocessing import QuantileTransformer

def umap_feature_creation(df, feature_cols, n_components=UMAP_N_COMPONENTS, random_state=42):
    """Creates features using UMAP."""
    print(f"Creating {n_components} UMAP features...")
    reducer = umap.UMAP(n_components=n_components, random_state=random_state)
    # Ensure data is float32 and handle potential NaNs (though features shouldn't have NaNs)
    umap_features = reducer.fit_transform(df[feature_cols].astype(np.float32).fillna(0.5))
    umap_feature_names = [f"umap_feat_{i}" for i in range(n_components)]
    df[umap_feature_names] = umap_features
    print("UMAP features created.")
    return df, umap_feature_names

def denoising_autoencoder_features(df, feature_cols, encoding_dim=AE_ENCODING_DIM, noise_factor=0.1, epochs=10, batch_size=256):
    """Creates features using a Denoising Autoencoder (Keras)."""
    print(f"Creating {encoding_dim} Denoising AE features...")
    input_dim = len(feature_cols)
    data = df[feature_cols].astype(np.float32).fillna(0.5).values

    # Add noise
    noisy_data = data + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=data.shape)
    noisy_data = np.clip(noisy_data, 0., 1.) # Assuming features are scaled 0-1

    # Define Autoencoder
    input_layer = keras.Input(shape=(input_dim,))
    encoded = layers.Dense(encoding_dim * 2, activation='relu')(input_layer)
    encoded = layers.Dense(encoding_dim, activation='relu')(encoded)
    decoded = layers.Dense(encoding_dim * 2, activation='relu')(encoded)
    decoded = layers.Dense(input_dim, activation='sigmoid')(decoded) # Sigmoid for [0, 1] output

    autoencoder = keras.Model(input_layer, decoded)
    encoder = keras.Model(input_layer, encoded)

    autoencoder.compile(optimizer='adam', loss='mse')

    # Train Autoencoder
    autoencoder.fit(noisy_data, data, # Train to reconstruct original from noisy
                    epochs=epochs,
                    batch_size=batch_size,
                    shuffle=True,
                    validation_split=0.1, # Use a small validation split
                    verbose=0) # Suppress verbose output for brevity

    # Get encoded features
    ae_features = encoder.predict(data)
    ae_feature_names = [f"ae_feat_{i}" for i in range(encoding_dim)]
    df[ae_feature_names] = ae_features
    print("AE features created.")
    return df, ae_feature_names

def contrastive_feature_creation(df, feature_cols, embedding_dim=CONTRASTIVE_EMB_DIM, epochs=5, batch_size=256):
    """Placeholder for contrastive learning feature creation (generates random embeddings)."""
    # NOTE: A proper implementation requires defining positive/negative pairs (e.g., based on eras or targets)
    # and training a model (like a Siamese network) with a contrastive loss (e.g., TripletLoss).
    # This placeholder generates random features for demonstration.
    print(f"Creating {embedding_dim} Contrastive (placeholder) features...")
    num_samples = len(df)
    contrastive_features = np.random.rand(num_samples, embedding_dim).astype(np.float32)
    contrastive_feature_names = [f"contrastive_feat_{i}" for i in range(embedding_dim)]
    df[contrastive_feature_names] = contrastive_features
    print("Contrastive (placeholder) features created.")
    return df, contrastive_feature_names

def synthetic_data_ctgan(df, feature_cols, target_col, n_synthetic_samples_ratio=0.5, epochs=CTGAN_EPOCHS):
    """Generates synthetic features using CTGAN based on original features and a target."""
    print(f"Creating synthetic features using CTGAN (target: {target_col})...")
    
    # CTGAN works best with discrete or transformed continuous data.
    # We'll use QuantileTransformer as features are somewhat ordinal.
    data_subset = df[feature_cols + [target_col]].copy().dropna(subset=[target_col])
    
    # Handle potential NaNs in features (though ideally preprocessed)
    data_subset[feature_cols] = data_subset[feature_cols].fillna(0.5)

    # Transform features to be more Gaussian-like (helps CTGAN)
    qt = QuantileTransformer(output_distribution='normal', random_state=42)
    data_transformed = qt.fit_transform(data_subset[feature_cols])
    data_transformed_df = pd.DataFrame(data_transformed, columns=feature_cols, index=data_subset.index)
    data_transformed_df[target_col] = data_subset[target_col].values # Add target back

    # Define discrete columns (none in this case, but important for CTGAN)
    discrete_columns = []

    # Train CTGAN
    ctgan = CTGAN(verbose=False)
    try:
        ctgan.fit(data_transformed_df, discrete_columns, epochs=epochs)
    except Exception as e:
        print(f"CTGAN fitting failed: {e}. Skipping CTGAN features.")
        return df, []

    # Generate synthetic samples
    n_synthetic_samples = int(len(data_subset) * n_synthetic_samples_ratio)
    synthetic_data_transformed = ctgan.sample(n_synthetic_samples)

    # Inverse transform the synthetic features
    synthetic_features_original_scale = qt.inverse_transform(synthetic_data_transformed[feature_cols])
    synthetic_df = pd.DataFrame(synthetic_features_original_scale, columns=feature_cols)
    
    # For simplicity, we'll create features based on the distance to synthetic samples.
    # A more advanced approach might use synthetic data for augmentation or training separate models.
    # Calculate mean of synthetic features as a simple derived feature
    synthetic_mean = synthetic_df.mean(axis=0)
    ctgan_feature_name = f"dist_to_synth_mean_{target_col}"
    
    # Calculate Euclidean distance for each original sample to the mean synthetic sample
    original_features = df[feature_cols].fillna(0.5).values
    distances = np.linalg.norm(original_features - synthetic_mean.values.astype(np.float32), axis=1)
    
    df[ctgan_feature_name] = distances
    print("CTGAN-derived features created.")
    
    del data_subset, data_transformed, data_transformed_df, synthetic_data_transformed, synthetic_features_original_scale, synthetic_df
    gc.collect()
    
    return df, [ctgan_feature_name]

print("Feature engineering functions defined.")

## 2. Data Loading and Initial Exploration

Load training data and explore auxiliary targets.

In [ ]:
napi = NumerAPI()

# Download metadata and training data
print("Downloading metadata...")
napi.download_dataset(f"{DATA_VERSION}/features.json")
print("Downloading training data...")
napi.download_dataset(f"{DATA_VERSION}/train.parquet")

# Load feature metadata and define feature sets
print("Loading feature metadata...")
feature_metadata = json.load(open(f"{DATA_VERSION}/features.json"))
feature_sets = feature_metadata["feature_sets"]
original_feature_cols = feature_sets["small"] # Start with small features
# original_feature_cols = feature_sets["medium"] # Use medium for potentially better performance
# original_feature_cols = feature_sets["all"] # Use all for potentially better performance
target_cols = feature_metadata["targets"]

# Load training data
print("Loading training data...")
train = pd.read_parquet(
    f"{DATA_VERSION}/train.parquet",
    columns=[ERA_COL, DATA_TYPE_COL] + original_feature_cols + target_cols
)

# Filter for training data type (just in case)
train = train[train[DATA_TYPE_COL] == "train"].copy()
del train[DATA_TYPE_COL]
gc.collect()

# Downsample eras if configured
if DOWNSAMPLE_TRAIN_ERAS > 1:
    print(f"Downsampling training data to every {DOWNSAMPLE_TRAIN_ERAS}th era...")
    train = train[train[ERA_COL].isin(train[ERA_COL].unique()[::DOWNSAMPLE_TRAIN_ERAS])]
    gc.collect()

# Display target columns
print("Target columns in training data:")
display(train[[ERA_COL] + target_cols].head())

### The Main Target

The primary target for Numerai predictions is typically `target_cyrusd_20`. The `target` column is often an alias for this.

In [ ]:
# Check if 'target' is an alias for the main target and prepare targets DataFrame
if TARGET_COL in train.columns:
    assert train[TARGET_COL].equals(train[MAIN_TARGET]), f"'{TARGET_COL}' column is not equal to '{MAIN_TARGET}'"
    print(f"'{TARGET_COL}' column confirmed as alias for '{MAIN_TARGET}'.")
    targets_df = train[[ERA_COL] + target_cols].drop(columns=[TARGET_COL])
    target_cols.remove(TARGET_COL)
else:
    targets_df = train[[ERA_COL] + target_cols]

print(f"Using '{MAIN_TARGET}' as the main target.")

### Target Names and Correlations

Auxiliary targets represent different stock market return definitions or time horizons (`_20` vs `_60` days). They have varying correlations with the main target, which can be useful for ensembling.

In [ ]:
# Print target names grouped by name and time horizon
t20s = sorted([t for t in target_cols if t.endswith("_20")])
t60s = sorted([t for t in target_cols if t.endswith("_60")])
names = sorted(list(set([t.replace("target_", "").replace("_20", "").replace("_60", "") for t in target_cols])))

target_map_df = pd.DataFrame(index=names, columns=['20', '60'])
for t in t20s:
    name = t.replace("target_", "").replace("_20", "")
    target_map_df.loc[name, '20'] = t
for t in t60s:
    name = t.replace("target_", "").replace("_60", "")
    target_map_df.loc[name, '60'] = t

print("Target names grouped by name and horizon:")
display(target_map_df)

In [ ]:
# Calculate and display correlations with the main target
print(f"Correlations of auxiliary targets with {MAIN_TARGET}:")
target_corrs = (
    targets_df[target_cols]
    .corrwith(targets_df[MAIN_TARGET])
    .sort_values(ascending=False)
    .to_frame(f"corr_with_{MAIN_TARGET}")
)
display(target_corrs)

# Plot correlation matrix heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(
  targets_df[target_cols].corr(),
  cmap="coolwarm",
  xticklabels=False,
  yticklabels=False
)
plt.title("Target Correlation Matrix")
plt.show()

## 3. Apply Feature Engineering

Generate UMAP, Denoising Autoencoder, Contrastive (placeholder), and CTGAN features for both training and validation data.

In [ ]:
# --- Apply Feature Engineering to Training Data ---
engineered_feature_cols = []

# UMAP
train, umap_feats = umap_feature_creation(train, original_feature_cols, n_components=UMAP_N_COMPONENTS)
engineered_feature_cols.extend(umap_feats)
gc.collect()

# Denoising Autoencoder
train, ae_feats = denoising_autoencoder_features(train, original_feature_cols, encoding_dim=AE_ENCODING_DIM)
engineered_feature_cols.extend(ae_feats)
gc.collect()

# Contrastive Learning (Placeholder)
train, contrastive_feats = contrastive_feature_creation(train, original_feature_cols, embedding_dim=CONTRASTIVE_EMB_DIM)
engineered_feature_cols.extend(contrastive_feats)
gc.collect()

# CTGAN (using main target for demonstration)
# Note: CTGAN can be computationally expensive
train, ctgan_feats = synthetic_data_ctgan(train, original_feature_cols, MAIN_TARGET, epochs=CTGAN_EPOCHS)
engineered_feature_cols.extend(ctgan_feats)
gc.collect()

# Update the main feature list
feature_cols = original_feature_cols + engineered_feature_cols
print(f"\nTotal number of features after engineering: {len(feature_cols)}")
print(f"Engineered feature names: {engineered_feature_cols}")

# --- Load and Apply Feature Engineering to Validation Data ---
print("\nDownloading validation data...")
napi.download_dataset(f"{DATA_VERSION}/validation.parquet")

print("Loading validation data...")
validation = pd.read_parquet(
    f"{DATA_VERSION}/validation.parquet",
    columns=[ERA_COL, DATA_TYPE_COL] + original_feature_cols + target_cols
)
validation = validation[validation[DATA_TYPE_COL] == "validation"].copy()
del validation[DATA_TYPE_COL]
gc.collect()

# Downsample validation eras if configured
if DOWNSAMPLE_VALID_ERAS > 1:
    print(f"Downsampling validation data to every {DOWNSAMPLE_VALID_ERAS}th era...")
    validation = validation[validation[ERA_COL].isin(validation[ERA_COL].unique()[::DOWNSAMPLE_VALID_ERAS])]
    gc.collect()

# Embargo overlapping eras
last_train_era = int(train[ERA_COL].unique()[-1])
eras_to_embargo = [str(era).zfill(4) for era in [last_train_era + i for i in range(1, 5)]] # Embargo 4 eras after last train era
validation = validation[~validation[ERA_COL].isin(eras_to_embargo)].copy()
print(f"Embargoed eras: {eras_to_embargo}")
gc.collect()

# Apply feature engineering transformations (using fitted models/transformers where applicable)
# UMAP (Refit or use transform - refitting for simplicity here, but transform is better practice)
validation, _ = umap_feature_creation(validation, original_feature_cols, n_components=UMAP_N_COMPONENTS)
gc.collect()

# Denoising Autoencoder (Use the trained encoder)
print("Applying AE transformation to validation data...")
val_data_ae = validation[original_feature_cols].astype(np.float32).fillna(0.5).values
ae_features_val = encoder.predict(val_data_ae)
validation[ae_feats] = ae_features_val
print("AE features applied to validation data.")
gc.collect()

# Contrastive Learning (Placeholder - generate random features)
validation, _ = contrastive_feature_creation(validation, original_feature_cols, embedding_dim=CONTRASTIVE_EMB_DIM)
gc.collect()

# CTGAN (Calculate distance feature for validation data)
if ctgan_feats: # Only if CTGAN succeeded on train data
    print("Applying CTGAN-derived feature calculation to validation data...")
    val_features_original = validation[original_feature_cols].fillna(0.5).values
    distances_val = np.linalg.norm(val_features_original - synthetic_mean.values.astype(np.float32), axis=1)
    validation[ctgan_feats[0]] = distances_val
    print("CTGAN-derived features applied to validation data.")
else:
    print("Skipping CTGAN features for validation data as it failed during training.")
gc.collect()

print("\nValidation data with engineered features:")
display(validation[feature_cols].head())

## 4. Base Model Training (LightGBM)

Train LightGBM models for each selected target using the original and engineered features.

In [ ]:
print("Training LightGBM models on selected targets...")
models = {}
for target in tqdm(TARGET_CANDIDATES, desc="Training models"):
    print(f"Training model for {target}...")
    # Filter out rows where the current target is NaN for training
    train_target_filtered = train.dropna(subset=[target])
    
    model = lgb.LGBMRegressor(
        n_estimators=2000,
        learning_rate=0.01,
        max_depth=5,
        num_leaves=2**4-1,
        colsample_bytree=0.1,
        random_state=42,
        n_jobs=-1
    )
    model.fit(
        train_target_filtered[feature_cols],
        train_target_filtered[target]
    )
    models[target] = model
    gc.collect()

print("Base models trained.")

### Base Model Evaluation

Generate predictions on the validation set for each base model and evaluate their individual performance (Correlation).

In [ ]:
from numerai_tools.scoring import numerai_corr

print("Generating validation predictions for base models...")
validation_preds = pd.DataFrame(index=validation.index)
for target_name, model in models.items():
    pred_col_name = f"prediction_{target_name}"
    validation_preds[pred_col_name] = model.predict(validation[feature_cols])

# Merge predictions back into the validation dataframe
validation = validation.join(validation_preds)

prediction_cols = list(validation_preds.columns)
print("\nValidation predictions generated:")
display(validation[prediction_cols].head())

# Evaluate individual model correlations
print("\nEvaluating base model correlations...")
correlations = validation.groupby(ERA_COL).apply(
    lambda d: numerai_corr(d[prediction_cols], d[MAIN_TARGET])
)
cumsum_corrs = correlations.cumsum()

plt.figure(figsize=(10, 6))
cumsum_corrs.plot(ax=plt.gca())
plt.title("Cumulative Correlation of Base Model Validation Predictions")
plt.xlabel("Era")
plt.ylabel("Cumulative Correlation")
plt.xticks([])
plt.legend(title="Model Target")
plt.grid(True, linestyle='--', alpha=0.5)
plt.show()

print("\nSummary metrics for base models:")
def get_summary_metrics(scores, cumsum_scores):
    summary_metrics = {}
    mean = scores.mean()
    std = scores.std()
    sharpe = mean / std if std != 0 else np.nan
    rolling_max = cumsum_scores.expanding(min_periods=1).max()
    max_drawdown = (rolling_max - cumsum_scores).max()
    return {
        "mean": mean,
        "std": std,
        "sharpe": sharpe,
        "max_drawdown": max_drawdown,
    }

base_model_summary = {}
for pred_col in prediction_cols:
    base_model_summary[pred_col] = get_summary_metrics(correlations[pred_col], cumsum_corrs[pred_col])

summary_df = pd.DataFrame(base_model_summary).T
display(summary_df)

## 5. Stacked Ensembling

Implement a stacked ensemble using out-of-fold predictions from the base models.

In [ ]:
print("Generating Out-of-Fold (OOF) predictions for stacking...")

gkf = GroupKFold(n_splits=N_FOLDS)
oof_preds = pd.DataFrame(index=train.index)

# Store OOF predictions for each base model
for target_name, model in tqdm(models.items(), desc="Generating OOF preds"):
    print(f" Generating OOF for {target_name}...")
    oof_preds_target = pd.Series(index=train.index, dtype=np.float32)
    train_target_filtered = train.dropna(subset=[target_name]) # Use only non-NaN target rows for training folds
    
    for fold, (train_index, val_index) in enumerate(gkf.split(train_target_filtered[feature_cols], train_target_filtered[target_name], groups=train_target_filtered[ERA_COL])):
        X_train_fold, X_val_fold = train_target_filtered.iloc[train_index][feature_cols], train_target_filtered.iloc[val_index][feature_cols]
        y_train_fold, y_val_fold = train_target_filtered.iloc[train_index][target_name], train_target_filtered.iloc[val_index][target_name]
        
        fold_model = lgb.LGBMRegressor(
            n_estimators=500, # Fewer estimators for fold training
            learning_rate=0.01,
            max_depth=5,
            num_leaves=2**4-1,
            colsample_bytree=0.1,
            random_state=fold, # Vary random state per fold
            n_jobs=-1
        )
        fold_model.fit(X_train_fold, y_train_fold)
        
        # Store predictions on the validation part of the fold, using original train index
        oof_preds_target.iloc[val_index] = fold_model.predict(X_val_fold)
        
    oof_preds[f"oof_{target_name}"] = oof_preds_target
    gc.collect()

print("OOF predictions generated.")
display(oof_preds.head())

# Prepare training data for the meta-model
meta_train_features = oof_preds.copy()
meta_train_features[ERA_COL] = train[ERA_COL] # Add era for potential use in meta-model
meta_train_target = train[MAIN_TARGET]

# Drop rows where OOF preds or target are NaN (can happen if target was NaN in original data)
valid_indices = meta_train_target.notna() & meta_train_features.notna().all(axis=1)
meta_train_features = meta_train_features[valid_indices]
meta_train_target = meta_train_target[valid_indices]

oof_feature_cols = list(oof_preds.columns)

# Train the meta-model
print(f"\nTraining meta-model ({STACKING_MODEL_TYPE})...")
if STACKING_MODEL_TYPE == 'LGBM':
    meta_model = lgb.LGBMRegressor(
        n_estimators=500,
        learning_rate=0.01,
        max_depth=3,
        num_leaves=2**3-1,
        colsample_bytree=0.8, # Use more features for meta-model
        random_state=42,
        n_jobs=-1
    )
    meta_model.fit(meta_train_features[oof_feature_cols], meta_train_target)
elif STACKING_MODEL_TYPE == 'Linear':
    scaler = StandardScaler()
    meta_train_features_scaled = scaler.fit_transform(meta_train_features[oof_feature_cols])
    meta_model = Ridge(alpha=1.0, random_state=42)
    meta_model.fit(meta_train_features_scaled, meta_train_target)
else:
    raise ValueError("Invalid STACKING_MODEL_TYPE")

print("Meta-model trained.")

# Generate predictions on the validation set using the stacking pipeline
print("\nGenerating stacked predictions on validation set...")
meta_val_features = validation[prediction_cols].copy()
meta_val_features.columns = oof_feature_cols # Rename columns to match meta-model training

if STACKING_MODEL_TYPE == 'Linear':
    meta_val_features_scaled = scaler.transform(meta_val_features)
    stacked_preds = meta_model.predict(meta_val_features_scaled)
else: # LGBM
    stacked_preds = meta_model.predict(meta_val_features)

validation["prediction_stacked"] = stacked_preds

print("Stacked predictions generated.")
display(validation[["prediction_stacked"]].head())

### Stacked Ensemble Evaluation

Evaluate the performance of the stacked ensemble.

In [ ]:
print("Evaluating stacked ensemble performance...")

# Add stacked predictions to the list for evaluation
evaluation_cols = prediction_cols + ["prediction_stacked"]

stacked_correlations = validation.groupby(ERA_COL).apply(
    lambda d: numerai_corr(d[evaluation_cols], d[MAIN_TARGET])
)
stacked_cumsum_corrs = stacked_correlations.cumsum()

plt.figure(figsize=(10, 6))
stacked_cumsum_corrs.plot(ax=plt.gca())
plt.title("Cumulative Correlation including Stacked Ensemble")
plt.xlabel("Era")
plt.ylabel("Cumulative Correlation")
plt.xticks([])
plt.legend(title="Model")
plt.grid(True, linestyle='--', alpha=0.5)
plt.show()

print("\nSummary metrics including Stacked Ensemble:")
stacked_summary = {}
for pred_col in evaluation_cols:
    stacked_summary[pred_col] = get_summary_metrics(stacked_correlations[pred_col], stacked_cumsum_corrs[pred_col])

stacked_summary_df = pd.DataFrame(stacked_summary).T
display(stacked_summary_df)

## 6. Era-Invariant Training (PyTorch MLP Option)

Define and train a PyTorch MLP with a custom loss function incorporating negative Pearson correlation, era correlation variance penalty, and feature exposure penalty.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from scipy.stats import rankdata

# --- Define MLP Architecture ---
class SimpleMLP(nn.Module):
    def __init__(self, input_dim):
        super(SimpleMLP, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(input_dim, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 1),
            nn.Sigmoid() # Output between 0 and 1
        )

    def forward(self, x):
        return self.layers(x)

# --- Define Custom Loss Functions ---
def pearson_corr(preds, target):
    """Calculate Pearson correlation coefficient."""
    preds_mean = torch.mean(preds)
    target_mean = torch.mean(target)
    cov = torch.mean((preds - preds_mean) * (target - target_mean))
    preds_std = torch.std(preds)
    target_std = torch.std(target)
    # Add epsilon to prevent division by zero
    epsilon = 1e-6
    corr = cov / (preds_std * target_std + epsilon)
    return corr

def era_correlation_variance_penalty(preds, target, eras):
    """Calculate variance of per-era correlations."""
    unique_eras = torch.unique(eras)
    era_corrs = []
    for era in unique_eras:
        era_mask = (eras == era)
        era_preds = preds[era_mask]
        era_target = target[era_mask]
        if len(era_preds) > 1: # Need at least 2 points for correlation
            era_corrs.append(pearson_corr(era_preds, era_target))
    
    if len(era_corrs) > 1:
        era_corrs_tensor = torch.stack(era_corrs)
        # Handle potential NaNs from zero std dev within an era
        valid_corrs = era_corrs_tensor[~torch.isnan(era_corrs_tensor)]
        if len(valid_corrs) > 1:
             return torch.var(valid_corrs)
    return torch.tensor(0.0, device=preds.device) # Return 0 if variance can't be calculated

def feature_exposure_penalty(preds, features):
    """Calculate sum of squared correlations between predictions and features."""
    num_features = features.shape[1]
    feature_corrs_sq = []
    for i in range(num_features):
        feature_col = features[:, i]
        if torch.std(feature_col) > 1e-6: # Check for constant features
             corr = pearson_corr(preds.squeeze(), feature_col)
             if not torch.isnan(corr):
                 feature_corrs_sq.append(corr**2)
        
    if len(feature_corrs_sq) > 0:
        return torch.mean(torch.stack(feature_corrs_sq))
    return torch.tensor(0.0, device=preds.device)

# --- Training Loop ---
def train_mlp(train_df, feature_cols, target_col, era_col, top_n_features=TOP_N_FEATURES_FOR_EXPOSURE):
    print("\nTraining PyTorch MLP with custom loss...")
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")

    # Prepare data
    train_target_filtered = train_df.dropna(subset=[target_col])
    features = torch.tensor(train_target_filtered[feature_cols].fillna(0.5).values, dtype=torch.float32).to(device)
    target = torch.tensor(train_target_filtered[target_col].values, dtype=torch.float32).unsqueeze(1).to(device)
    eras = torch.tensor(train_target_filtered[era_col].astype(int).values, dtype=torch.long).to(device)
    
    # Select top N features for feature exposure penalty (based on correlation with target)
    feature_corrs = train_target_filtered[feature_cols].corrwith(train_target_filtered[target_col])
    top_feature_indices = feature_corrs.abs().nlargest(top_n_features).index
    top_features_tensor = torch.tensor(train_target_filtered[top_feature_indices].fillna(0.5).values, dtype=torch.float32).to(device)
    print(f"Using top {len(top_feature_indices)} features for exposure penalty.")

    dataset = TensorDataset(features, target, eras, top_features_tensor)
    dataloader = DataLoader(dataset, batch_size=MLP_BATCH_SIZE, shuffle=True)

    # Initialize model, optimizer, loss weights
    input_dim = len(feature_cols)
    mlp_model = SimpleMLP(input_dim).to(device)
    optimizer = optim.Adam(mlp_model.parameters(), lr=MLP_LR)
    lambda1 = VARIANCE_PENALTY_WEIGHT
    lambda2 = FEATURE_EXPOSURE_WEIGHT

    # Training
    mlp_model.train()
    for epoch in range(MLP_EPOCHS):
        epoch_loss = 0.0
        for batch_features, batch_target, batch_eras, batch_top_features in dataloader:
            optimizer.zero_grad()
            preds = mlp_model(batch_features)

            # Calculate loss components
            corr_loss = -pearson_corr(preds, batch_target)
            var_penalty = era_correlation_variance_penalty(preds, batch_target, batch_eras)
            exposure_penalty = feature_exposure_penalty(preds, batch_top_features)

            # Combine losses
            total_loss = corr_loss + lambda1 * var_penalty + lambda2 * exposure_penalty
            
            # Handle potential NaN loss
            if torch.isnan(total_loss):
                print(f"Warning: NaN loss encountered in epoch {epoch+1}. Skipping batch.")
                continue

            total_loss.backward()
            optimizer.step()
            epoch_loss += total_loss.item()
        
        avg_epoch_loss = epoch_loss / len(dataloader)
        print(f"Epoch [{epoch+1}/{MLP_EPOCHS}], Loss: {avg_epoch_loss:.6f}")
        
    print("MLP training finished.")
    return mlp_model.to('cpu') # Move model back to CPU for prediction

# --- Control Flow for Model Training ---
mlp_model = None
if USE_MLP:
    # Ensure PyTorch and dependencies are installed
    try:
        import torch
        mlp_model = train_mlp(train, feature_cols, MAIN_TARGET, ERA_COL)
        
        # Generate MLP predictions on validation set
        print("Generating MLP predictions on validation set...")
        mlp_model.eval()
        with torch.no_grad():
            val_features_tensor = torch.tensor(validation[feature_cols].fillna(0.5).values, dtype=torch.float32)
            mlp_preds = mlp_model(val_features_tensor).numpy().squeeze()
        validation["prediction_mlp"] = mlp_preds
        print("MLP predictions generated.")
        display(validation[["prediction_mlp"]].head())

    except ImportError:
        print("PyTorch not found. Skipping MLP training. Set USE_MLP=False or install PyTorch.")
    except Exception as e:
        print(f"An error occurred during MLP training or prediction: {e}")
        mlp_model = None # Ensure mlp_model is None if training failed
elif not USE_STACKING:
    print("Skipping Stacking and MLP training based on configuration.")
    print("Using the base model for the main target as the final prediction.")
    validation["prediction_final"] = validation[f"prediction_{MAIN_TARGET}"]
else:
    print("Using Stacked Ensemble based on configuration.")
    validation["prediction_final"] = validation["prediction_stacked"]

## 7. Final Model Evaluation

Evaluate the chosen final model (either Stacked Ensemble or MLP).

In [ ]:
print("Evaluating final model performance...")

final_pred_col = "prediction_mlp" if USE_MLP and mlp_model else ("prediction_stacked" if USE_STACKING else f"prediction_{MAIN_TARGET}")

if final_pred_col not in validation.columns:
    print(f"Warning: Final prediction column '{final_pred_col}' not found. Defaulting to main target model.")
    final_pred_col = f"prediction_{MAIN_TARGET}"
    if final_pred_col not in validation.columns:
         raise ValueError("Could not find a valid prediction column for final evaluation.")

print(f"Evaluating column: {final_pred_col}")

final_evaluation_cols = [final_pred_col]
if USE_STACKING:
    final_evaluation_cols = evaluation_cols # Show comparison if stacking was run
elif USE_MLP and mlp_model:
    final_evaluation_cols = [f"prediction_{MAIN_TARGET}"] + [final_pred_col] # Compare MLP to base


final_correlations = validation.groupby(ERA_COL).apply(
    lambda d: numerai_corr(d[final_evaluation_cols], d[MAIN_TARGET])
)
final_cumsum_corrs = final_correlations.cumsum()

plt.figure(figsize=(10, 6))
final_cumsum_corrs.plot(ax=plt.gca())
plt.title(f"Cumulative Correlation of Final Model ({final_pred_col}) vs Others")
plt.xlabel("Era")
plt.ylabel("Cumulative Correlation")
plt.xticks([])
plt.legend(title="Model")
plt.grid(True, linestyle='--', alpha=0.5)
plt.show()

print("\nFinal Summary Metrics:")
final_summary = {}
for pred_col in final_evaluation_cols:
    final_summary[pred_col] = get_summary_metrics(final_correlations[pred_col], final_cumsum_corrs[pred_col])

final_summary_df = pd.DataFrame(final_summary).T
display(final_summary_df)

## 8. Model Upload

Define the final prediction function based on the selected model (Stacking or MLP) and prepare for upload.

In [ ]:
# --- Define Final Prediction Function ---
def predict_final(live_features: pd.DataFrame) -> pd.DataFrame:
    """Generates predictions using the chosen final model."""
    
    # Apply feature engineering transformations to live features
    # Important: Use transform methods of fitted transformers (UMAP, AE encoder, Scaler) 
    # and pre-calculated values (CTGAN synthetic mean) from training.
    # This requires saving these objects during training.
    # For this example, we'll re-run the generation functions, which is NOT best practice 
    # for production but simpler for this notebook structure.
    print("Applying feature engineering to live data...")
    live_features_eng = live_features.copy()
    live_features_eng, _ = umap_feature_creation(live_features_eng, original_feature_cols, n_components=UMAP_N_COMPONENTS)
    
    live_data_ae = live_features_eng[original_feature_cols].astype(np.float32).fillna(0.5).values
    ae_features_live = encoder.predict(live_data_ae)
    live_features_eng[ae_feats] = ae_features_live
    
    live_features_eng, _ = contrastive_feature_creation(live_features_eng, original_feature_cols, embedding_dim=CONTRASTIVE_EMB_DIM)
    
    if ctgan_feats: # Only if CTGAN succeeded on train data
        live_features_original = live_features_eng[original_feature_cols].fillna(0.5).values
        distances_live = np.linalg.norm(live_features_original - synthetic_mean.values.astype(np.float32), axis=1)
        live_features_eng[ctgan_feats[0]] = distances_live
    
    print("Feature engineering applied to live data.")

    if USE_MLP and mlp_model:
        print("Generating predictions using MLP model...")
        mlp_model.eval()
        with torch.no_grad():
            live_features_tensor = torch.tensor(live_features_eng[feature_cols].fillna(0.5).values, dtype=torch.float32)
            predictions = mlp_model(live_features_tensor).numpy().squeeze()
        submission_df = pd.DataFrame({'prediction': predictions}, index=live_features.index)
        
    elif USE_STACKING:
        print("Generating predictions using Stacked Ensemble...")
        # Generate base model predictions on live data
        base_preds_live = pd.DataFrame(index=live_features.index)
        for target_name, model in models.items():
            base_preds_live[f"oof_{target_name}"] = model.predict(live_features_eng[feature_cols])
        
        # Generate meta-model predictions
        if STACKING_MODEL_TYPE == 'Linear':
             base_preds_live_scaled = scaler.transform(base_preds_live)
             stacked_preds_live = meta_model.predict(base_preds_live_scaled)
        else: # LGBM
             stacked_preds_live = meta_model.predict(base_preds_live)
        
        submission_df = pd.DataFrame({'prediction': stacked_preds_live}, index=live_features.index)

    else: # Default to main target base model if others are disabled/failed
        print(f"Generating predictions using base model for {MAIN_TARGET}...")
        predictions = models[MAIN_TARGET].predict(live_features_eng[feature_cols])
        submission_df = pd.DataFrame({'prediction': predictions}, index=live_features.index)

    # Rank predictions for submission
    ranked_submission = submission_df['prediction'].rank(pct=True, method="first")
    return ranked_submission.to_frame(PREDICTION_COL)

In [ ]:
# --- Quick Test on Live Data ---
print("Downloading live features for testing...")
napi.download_dataset(f"{DATA_VERSION}/live.parquet")
live_features = pd.read_parquet(f"{DATA_VERSION}/live.parquet", columns=original_feature_cols)

# Generate predictions using the final function
final_predictions = predict_final(live_features)

print("\nSample of final predictions:")
display(final_predictions.head())

In [ ]:
# --- Pickle the Prediction Function and Dependencies ---
# Important: Ensure all necessary objects (models, transformers, etc.) 
# are either defined within predict_final or are globally accessible 
# AND will be pickled by cloudpickle.

# For this example, we rely on global models, encoder, scaler, synthetic_mean
# A more robust approach would pass these as arguments or load them within the function.

print("Pickling the prediction function...")
try:
    # Ensure necessary global objects are available if not defined inside predict_final
    global_dependencies = {
        'models': models,
        'encoder': encoder if 'encoder' in globals() else None, # Keras AE encoder
        'scaler': scaler if STACKING_MODEL_TYPE == 'Linear' and 'scaler' in globals() else None, # Scaler for linear meta-model
        'meta_model': meta_model if USE_STACKING and 'meta_model' in globals() else None, # Stacking meta-model
        'mlp_model': mlp_model if USE_MLP and mlp_model else None, # PyTorch MLP model
        'original_feature_cols': original_feature_cols,
        'engineered_feature_cols': engineered_feature_cols,
        'feature_cols': feature_cols,
        'ae_feats': ae_feats if 'ae_feats' in globals() else [],
        'contrastive_feats': contrastive_feats if 'contrastive_feats' in globals() else [],
        'ctgan_feats': ctgan_feats if 'ctgan_feats' in globals() else [],
        'synthetic_mean': synthetic_mean if 'synthetic_mean' in globals() and ctgan_feats else None, # Mean from CTGAN
        'UMAP_N_COMPONENTS': UMAP_N_COMPONENTS,
        'AE_ENCODING_DIM': AE_ENCODING_DIM,
        'CONTRASTIVE_EMB_DIM': CONTRASTIVE_EMB_DIM,
        'USE_MLP': USE_MLP,
        'USE_STACKING': USE_STACKING,
        'STACKING_MODEL_TYPE': STACKING_MODEL_TYPE,
        'MAIN_TARGET': MAIN_TARGET,
        'PREDICTION_COL': PREDICTION_COL
    }
    
    # Include the feature engineering functions themselves if they are not standard library imports
    global_dependencies['umap_feature_creation'] = umap_feature_creation
    global_dependencies['denoising_autoencoder_features'] = denoising_autoencoder_features
    global_dependencies['contrastive_feature_creation'] = contrastive_feature_creation
    global_dependencies['synthetic_data_ctgan'] = synthetic_data_ctgan
    
    # Pickle the function along with its dependencies
    cloudpickle.register_pickle_by_value(umap)
    cloudpickle.register_pickle_by_value(tf)
    cloudpickle.register_pickle_by_value(torch)
    cloudpickle.register_pickle_by_value(ctgan)
    
    with open("predict_final_model.pkl", "wb") as f:
        cloudpickle.dump({'predict_fn': predict_final, 'dependencies': global_dependencies}, f)
    print("Prediction function pickled successfully to predict_final_model.pkl")

except NameError as e:
    print(f"Pickling failed: A required object might not be defined. Error: {e}")
    print("Ensure all models and transformers used in 'predict_final' are trained and available globally.")
except Exception as e:
     print(f"An unexpected error occurred during pickling: {e}")

In [ ]:
# Download file if running in Google Colab
try:
    from google.colab import files
    files.download('predict_final_model.pkl')
except ImportError:
    print("Skipping download (not in Colab environment).")
except Exception as e:
    print(f"File download failed: {e}")

## 9. Conclusion

This notebook demonstrated adding feature engineering, stacked ensembling, and an optional era-invariant MLP training pipeline to the original target ensembling notebook.

Remember to choose the model (Stacking or MLP) you want to submit by setting the `USE_STACKING` or `USE_MLP` flags before pickling and uploading `predict_final_model.pkl` to [numer.ai](https://numer.ai).